# Network Security — Exercise 5: Domain Name System Security & Network Firewalls

**Noah Link, Jan Pfeifer, Julian Weske**  
**Date:** 20.06.2024

## Domain Name System Security (Time spent: xx h)

### Investigating Legal Websites

Initializing the project and dnsdbq worked just fine:
```
./dnsdbq -u circl -n www.heise.de
;; record times: 2023-10-08 18:53:22 .. 2024-06-12 16:29:03 (~247d 21h 35m)
;; count: 369
193.99.144.85  A  www.heise.de

... skipped ...

;; record times: 2024-01-21 15:44:12 .. 2024-06-11 23:03:38 (~142d 7h 19m)
;; count: 7
www.heise.de  PTR  85.144.99.193.in-addr.arpa
```

### Passive DNS

- **What is passive DNS?**  
  Passive DNS is a method of collecting, storing, and analyzing DNS query and response data from recursive DNS servers, allowing for historical tracking of domain-to-IP mappings without actively querying DNS servers.

- **How can investigations in cybercrime benefit from passive DNS analysis?**
  - Identifying historical domain-IP mappings to uncover the infrastructure used by cybercriminals.
  - Tracking changes in domain associations, which can reveal patterns of malicious activity and help in mapping out criminal networks.

- **Name two factors that the quality of passive DNS analysis, i.e., the number of returned results, depends on.**
  - The volume and diversity of DNS data sources contributing to the passive DNS database.
  - The time span over which DNS data has been collected and stored.

## IT Security News

In [69]:
import subprocess
import re
import socket

def query_dnsdbq(query_type, query_value):
    command = ["./source_dnsdbq/dnsdbq", "-u", "circl", f"-{query_type}", query_value]

    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        output = result.stdout.strip().split('\n')
        return output
    except subprocess.CalledProcessError as e:
        print(f"Error running dnsdbq: {e}")
        print(f"Standard Error: {e.stderr}")
        return None

def query_ptr_records(ip_address):
    try:
        fqdns = socket.gethostbyaddr(ip_address)[0]
        return fqdns
    except socket.herror as e:
        if e.errno == socket.herror.ENOENT:
            print(f"No PTR record found for IP address {ip_address}")
        elif e.errno == socket.herror.EAI_NONAME:
            print(f"No hostname associated with IP address {ip_address}")
        else:
            print(f"Socket error occurred for IP address {ip_address}: {e}")
        return None
    except UnicodeError as e:
        print(f"Unicode error occurred while decoding hostname for IP address {ip_address}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred for IP address {ip_address}: {e}")
        return None

def query_aaaa_records(query_value):
    command = ["./source_dnsdbq/dnsdbq", "-u", "circl", "-r", query_value]

    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        ipv6_addresses = re.findall(
            r"\b(?:[0-9a-fA-F]{1,4}:){7}[0-9a-fA-F]{1,4}\b",
            result.stdout
        )

        return ipv6_addresses
    except subprocess.CalledProcessError as e:
        print(f"Error running dnsdbq for AAAA records: {e}")
        print(f"Standard Error: {e.stderr}")
        return None

def get_ipv4_addresses(domain):
    # Query for A records initially
    a_records = query_dnsdbq("n", domain)

    if not a_records:
        print(f"No A records found for {domain}")
        return []

    # Extract A records (IPv4 addresses)
    ipv4_addresses = []
    for record in a_records:
        if re.match(r"^\d+\.\d+\.\d+\.\d+$", record):
            ipv4_addresses.append(record)

    # Check if any CNAME records found and follow them
    if not ipv4_addresses:
        print(f"Found CNAME records instead of A records for {domain}. Following CNAME chain.")

        # Find the most recent alias name (CNAME)
        recent_cname = a_records[-1].strip()

        # Query for A records of the most recent alias name
        ipv4_addresses = query_dnsdbq("n", recent_cname)

    return ipv4_addresses

def get_cname_chain(domain):
    cname_chain = []
    current_domain = domain

    while True:
        # Query CNAME records for current domain
        cname_records = query_dnsdbq("c", current_domain)
        
        if not cname_records:
            print(f"No CNAME records found for {current_domain}")
            break
        
        # Extract the most recent CNAME (last line in output)
        most_recent_cname = cname_records[-1].strip()

        # Add to chain and update current domain to follow next CNAME
        cname_chain.append(most_recent_cname)
        current_domain = most_recent_cname

        # Check if the most recent CNAME is the final target (no more CNAMEs)
        if len(cname_records) == 1:
            break
    
    return cname_chain

1. How many IPv4 addresses exist for this site? List all addresses (i.e., A records)
you found:

In [70]:
# 1.1.2.1
query_type = "n"
query_value = "www.heise.de"
ipv4_addresses = query_dnsdbq(query_type, query_value)

if ipv4_addresses:
    print(f"Number of IPv4 addresses found for {query_value}: {len(ipv4_addresses)}")
    print("IPv4 Addresses:")
    for address in ipv4_addresses:
        print(address)
else:
    print("No IPv4 addresses found or there was an error.")

Number of IPv4 addresses found for www.heise.de: 59
IPv4 Addresses:
;; record times: 2023-10-08 18:53:22 .. 2024-06-12 16:29:03 (~247d 21h 35m)
;; count: 369
193.99.144.85  A  www.heise.de

;; record times: 2023-10-11 18:37:03 .. 2024-06-13 08:45:43 (~245d 14h 8m)
;; count: 174
2a02:2e0:3fe:1001:7777:772e:2:85  AAAA  www.heise.de

;; record times: 2023-10-10 14:11:27 .. 2024-06-13 08:45:43 (~246d 18h 34m)
;; count: 72
www.heise.de  CNAME  www.telepolis.de

;; record times: 2023-11-03 19:43:52 .. 2024-02-15 23:10:27 (~104d 3h 26m)
;; count: 9
www.heise.de  CNAME  karriere.heise-gruppe.de

;; record times: 2024-01-30 19:29:54 .. 2024-05-30 06:17:47 (~120d 10h 47m)
;; count: 3
www.heise.de  CNAME  compaliate.heise.de

;; record times: 2024-02-16 17:05:03 .. 2024-02-16 17:05:03 (1s)
;; count: 1
www.heise.de  CNAME  monocle.heise.de

;; record times: 2024-04-09 08:06:38 .. 2024-04-09 08:06:38 (1s)
;; count: 1
www.heise.de  CNAME  www.heisegroup.de

;; record times: 2023-10-09 00:36:15 .. 20

2. Is this site using IPv6 as well? If yes, list all addresses (i.e., AAAA records) you
found.

In [71]:
# 1.1.2.2
ipv6_addresses = query_aaaa_records(query_value)

if ipv6_addresses:
    print(f"\nNumber of IPv6 addresses found for {query_value}: {len(ipv6_addresses)}")
    print("IPv6 Addresses:")
    for address in ipv6_addresses:
        print(address)
else:
    print("No IPv6 addresses found or there was an error.")


Number of IPv6 addresses found for www.heise.de: 1
IPv6 Addresses:
2a02:2e0:3fe:1001:7777:772e:2:85


3. Search for other Fully-Qualified Domain Names (FQDNs) that are also hosted on
the first IPv4 address of www.heise.de.

In [72]:
# 1.1.2.3
if ipv4_addresses:
    first_ipv4_address = ipv4_addresses[0]
    print(
        f"\nFinding other FQDNs hosted on the first IPv4 address ({first_ipv4_address}) of {query_value}:"
    )
    ptr_record = query_ptr_records(first_ipv4_address)
    if ptr_record:
        print(f"\nFQDN for IP address {first_ipv4_address}:")
        print(ptr_record)
    else:
        print(f"No PTR record found for IP address {first_ipv4_address}")


Finding other FQDNs hosted on the first IPv4 address (;; record times: 2023-10-08 18:53:22 .. 2024-06-12 16:29:03 (~247d 21h 35m)) of www.heise.de:
Unicode error occurred while decoding hostname for IP address ;; record times: 2023-10-08 18:53:22 .. 2024-06-12 16:29:03 (~247d 21h 35m): encoding with 'idna' codec failed (UnicodeError: label empty or too long)
No PTR record found for IP address ;; record times: 2023-10-08 18:53:22 .. 2024-06-12 16:29:03 (~247d 21h 35m)


4. List all FQDNs that share a common IPv6 address with www.heise.de.

In [73]:
# 1.1.2.4
if ipv6_addresses:
    print(f"\nFinding other FQDNs that share a common IPv6 address with {query_value}:")
    common_fqdns = set()
    for ipv6_address in ipv6_addresses:
        associated_fqdns = query_ptr_records(ipv6_address)
        if associated_fqdns:
            common_fqdns.update(associated_fqdns.split())
            print(f"\nIPv6 Address: {ipv6_address}")
            print("Associated FQDNs:")
            for fqdn in associated_fqdns.split():
                print(fqdn)

    if common_fqdns:
        print("\nCommon FQDNs:")
        for fqdn in common_fqdns:
            print(fqdn)
    else:
        print("No common FQDNs found with associated IPv6 addresses.")


Finding other FQDNs that share a common IPv6 address with www.heise.de:

IPv6 Address: 2a02:2e0:3fe:1001:7777:772e:2:85
Associated FQDNs:
www.heise.de

Common FQDNs:
www.heise.de


### University Website

1. How many IPv4 addresses exist for this site? List all addresses (i.e., A records) you found.

In [74]:
# 1.1.3.1
query_value = "www.uni-hamburg.de"
ipv4_addresses = get_ipv4_addresses(query_value)

if ipv4_addresses:
    print(f"Number of IPv4 addresses found for {query_value}: {len(ipv4_addresses)}")
    print("IPv4 Addresses:")
    for address in ipv4_addresses:
        print(address)
else:
    print(f"No IPv4 addresses found for {query_value} or there was an error.")

Found CNAME records instead of A records for www.uni-hamburg.de. Following CNAME chain.
Number of IPv4 addresses found for www.uni-hamburg.de: 1
IPv4 Addresses:



2. Is this site using IPv6 as well? If yes, list all addresses (i.e., AAAA records) you found.

In [75]:
# 1.1.3.2
ipv6_addresses = query_aaaa_records(query_value)

if ipv6_addresses:
    print(f"\nNumber of IPv6 addresses found for {query_value}: {len(ipv6_addresses)}")
    print("IPv6 Addresses:")
    for address in ipv6_addresses:
        print(address)
else:
    print(f"No IPv6 addresses found for {query_value} or there was an error.")

No IPv6 addresses found for www.uni-hamburg.de or there was an error.



3. Search for other Fully-Qualified Domain Names (FQDNs) that are also hosted on the first IPv4 address of www.heise.de.

In [76]:
# 1.1.3.3
if ipv4_addresses:
    first_ipv4_address = ipv4_addresses[0]
    print(
        f"\nFinding other FQDNs hosted on the first IPv4 address ({first_ipv4_address}) of {query_value}:"
    )
    ptr_records = query_ptr_records(first_ipv4_address)
    if ptr_records:
        if isinstance(ptr_records, list):
            for ptr_record in ptr_records:
                print(f"\nFQDN for IP address {first_ipv4_address}:")
                print(ptr_record)
        else:
            print(f"\nFQDN for IP address {first_ipv4_address}:")
            print(ptr_records)
    else:
        print(f"No PTR record found for IP address {first_ipv4_address}")


Finding other FQDNs hosted on the first IPv4 address () of www.uni-hamburg.de:
An unexpected error occurred for IP address : wildcard resolved to multiple address
No PTR record found for IP address 



4. List all FQDNs that share a common IPv6 address with www.heise.de.

In [77]:
# 1.1.3.4
if ipv6_addresses:
    print(f"\nFinding other FQDNs that share a common IPv6 address with {query_value}:")
    common_fqdns = set()
    for ipv6_address in ipv6_addresses:
        associated_fqdns = query_ptr_records(ipv6_address)
        if associated_fqdns:
            common_fqdns.update(associated_fqdns.split())
            print(f"\nIPv6 Address: {ipv6_address}")
            print("Associated FQDNs:")
            for fqdn in associated_fqdns.split():
                print(fqdn)

    if common_fqdns:
        print("\nCommon FQDNs:")
        for fqdn in common_fqdns:
            print(fqdn)
    else:
        print("No common FQDNs found with associated IPv6 addresses.")

5. Find more domain names that do not directly resolve to the IP address of www.uni-hamburg.de but also indirectly via the respective CNAME.

In [78]:
# 1.1.3.5
print(f"\nFinding more domain names indirectly resolving to {query_value} via CNAME chains:")

cname_chain = get_cname_chain(query_value)
if cname_chain:
    print("\nCNAME Chain:")
    for cname in cname_chain:
        print(cname)
else:
    print(f"No CNAME chain found for {query_value}")


Finding more domain names indirectly resolving to www.uni-hamburg.de via CNAME chains:
Error running dnsdbq: Command '['./source_dnsdbq/dnsdbq', '-u', 'circl', '-c', 'www.uni-hamburg.de']' returned non-zero exit status 1.
Standard Error: error: there are no non-option arguments to this program

try   dnsdbq -h   for a short description of program usage.

No CNAME records found for www.uni-hamburg.de
No CNAME chain found for www.uni-hamburg.de


### Investigating Illegal Websites

In [82]:
import socket

def generate_ip_addresses(base_ip, subnet_mask):
    ip_addresses = []
    for i in range(subnet_mask + 1):
        ip = base_ip + '.' + str(i)
        ip_addresses.append(ip)
    return ip_addresses

def find_fqdns_in_subnet(base_ip, subnet_mask):
    ip_addresses = generate_ip_addresses(base_ip, subnet_mask)
    fqdns = {}
    for ip in ip_addresses:
        try:
            host = socket.gethostbyaddr(ip)
            fqdns[ip] = host[0]
        except socket.herror:
            fqdns[ip] = "Unknown host"
        except socket.gaierror as e:
            fqdns[ip] = f"Failed to resolve: {str(e)}"
    return fqdns

1. Search for FQDNs that are hosted within the address block 104.28.21.0/24. Describe your working steps.

In [83]:
# 1.2.1
base_ip = '104.28.21'
subnet_mask = 255

fqdns = find_fqdns_in_subnet(base_ip, subnet_mask)

for ip, fqdn in fqdns.items():
    print(f"IP: {ip} resolves to FQDN: {fqdn}")


IP: 104.28.21.0 resolves to FQDN: Unknown host
IP: 104.28.21.1 resolves to FQDN: Unknown host
IP: 104.28.21.2 resolves to FQDN: Unknown host
IP: 104.28.21.3 resolves to FQDN: Unknown host
IP: 104.28.21.4 resolves to FQDN: Unknown host
IP: 104.28.21.5 resolves to FQDN: Unknown host
IP: 104.28.21.6 resolves to FQDN: Unknown host
IP: 104.28.21.7 resolves to FQDN: Unknown host
IP: 104.28.21.8 resolves to FQDN: Unknown host
IP: 104.28.21.9 resolves to FQDN: Unknown host
IP: 104.28.21.10 resolves to FQDN: Unknown host
IP: 104.28.21.11 resolves to FQDN: Unknown host
IP: 104.28.21.12 resolves to FQDN: Unknown host
IP: 104.28.21.13 resolves to FQDN: Unknown host
IP: 104.28.21.14 resolves to FQDN: Unknown host
IP: 104.28.21.15 resolves to FQDN: Unknown host
IP: 104.28.21.16 resolves to FQDN: Unknown host
IP: 104.28.21.17 resolves to FQDN: Unknown host
IP: 104.28.21.18 resolves to FQDN: Unknown host
IP: 104.28.21.19 resolves to FQDN: Unknown host
IP: 104.28.21.20 resolves to FQDN: Unknown host
IP

2. Come up with at least four more appropriate search strings for filtering the FQDNs.

3. Apply your search strings to the full list of FQDNs and list the filtered names.

## DNS and Firewall Evasion (Time spent: xx h)



### DNS Mechanisms and Evasion Techniques

1. Why is DNS often used to bypass firewalls, and why is this a popular attack vector?

DNS is commonly used to bypass firewalls because it operates over port 53, which is often allowed through firewalls for essential network functionality. This makes DNS traffic a convenient channel for attackers to tunnel malicious data, evade traditional firewall rules, and establish covert communication channels through techniques like DNS tunneling and domain generation algorithms (DGAs). DNS queries can be used to bypass content filters that rely on domain names, enabling access to blocked content or domains. The difficulty in inspecting DNS traffic thoroughly without specialized tools also contributes to its popularity as an attack vector. Organizations must implement robust DNS monitoring and filtering solutions to mitigate these risks effectively.

2. Explain the process of how a DNS tunnel works from the client request through to the response.

A DNS tunnel works by encoding data into DNS queries sent by a client to a DNS server. The server extracts this data from the queries and sends back responses that contain the encoded information or acknowledgments. The client then decodes the data from the server's responses, enabling covert communication through DNS channels. This method bypasses firewalls and other security measures that may not inspect DNS traffic thoroughly for embedded content.

3. Describe a method to further cloak traffic via DNS tunneling. Provide a detailed description and analyze the overhead involved with concrete numbers and percentages.

To cloak traffic via DNS tunneling and bypass firewalls and IDS systems:

1. Payload Encryption: Encrypt data using AES-256 before embedding it into DNS queries to prevent interception.
2. Traffic Obfuscation: Randomize query structure, timing, and data chunk order to evade pattern detection. Introduce random delays of 50-200 milliseconds between queries.
3. Compression Techniques: Use compression to reduce query size and improve transmission efficiency, despite potential CPU overhead of 10-30%.
4. Query Name Encoding: Encode data in DNS query names using base32 or base64 to utilize alphanumeric characters valid in DNS.

Overhead Analysis:
- Encryption: Adds a 14% overhead due to padding and metadata.
- Compression: Introduces 10-30% CPU usage overhead and variable compression efficiency.
- Obfuscation: Increases latency and network utilization by 10-20% with random delays.
- Data Rate: Depending on conditions, achieves data transmission rates ranging from 240 to 3,000 bytes per second.


### DoH, DoT Implementation and Analysis

1. Implement a subset of DNS over HTTPS (DoH) and DNS over TLS (DoT) to query an A record. Demonstrate the implementation by querying a public DNS server.

1.1 DNS over HTTPS Implementation:

In [17]:
import requests

def query_doh(domain):
    doh_url = "https://cloudflare-dns.com/dns-query"
    headers = {"Accept": "application/dns-json"}
    params = {"name": domain, "type": "A"}

    try:
        response = requests.get(doh_url, headers=headers, params=params)
        response.raise_for_status()
        answer = response.json()
        if "Answer" in answer:
            return answer["Answer"]
        else:
            print("No 'Answer' field in the response.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

domain = "example.com"
result = query_doh(domain)
if result:
    for record in result:
        print(f"{record['name']} has A record {record['data']}")
else:
    print("No A record found.")


example.com has A record 93.184.215.14


1.2 DNS over TLS Implementation:

In [22]:
import socket
import ssl
import struct

def build_query(domain):
    # Header
    transaction_id = b"\xaa\xbb"
    flags = b"\x01\x00"
    questions = b"\x00\x01"
    answer_rrs = b"\x00\x00"
    authority_rrs = b"\x00\x00"
    additional_rrs = b"\x00\x00"
    header = (
        transaction_id + flags + questions + answer_rrs + authority_rrs + additional_rrs
    )

    # Question
    qname = (
        b"".join(
            (
                len(part).to_bytes(1, byteorder="big") + part.encode()
                for part in domain.split(".")
            )
        )
        + b"\x00"
    )
    qtype = b"\x00\x01"  # A record
    qclass = b"\x00\x01"  # IN
    question = qname + qtype + qclass

    return header + question

def parse_response(response, domain):
    try:
        # Unpack the header
        transaction_id, flags, questions, answer_rrs, authority_rrs, additional_rrs = struct.unpack("!6H", response[:12])
        print("Header Info:", transaction_id, flags, questions, answer_rrs, authority_rrs, additional_rrs)

        answer_start = 12 + len(domain.split('.')) + 2 + 4  # Skipping the question part
        answer = response[answer_start:]

        answers = []
        while answer:
            name, type, class_, ttl, data_len = struct.unpack("!HHHIH", answer[:12])
            address = answer[12:12 + data_len]
            answers.append(socket.inet_ntoa(address))
            answer = answer[12 + data_len:]

        return answers
    except struct.error as e:
        print(f"Failed to parse response: {e}")
        return None

def query_dot(domain):
    server = "1.1.1.1"
    port = 853
    buffer_size = 4096  # Increased buffer size

    query = build_query(domain)

    context = ssl.create_default_context()
    try:
        with socket.create_connection((server, port)) as sock:
            with context.wrap_socket(sock, server_hostname=server) as ssock:
                print("Sending query...")
                ssock.sendall(query)
                response = ssock.recv(buffer_size)
                print("Response received.")
                print("Raw response:", response)
                return parse_response(response, domain)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

domain = "example.com"
result = query_dot(domain)
if result:
    for record in result:
        print(f"{domain} has A record {record}")
else:
    print("No A record found or an error occurred.")


Sending query...
Response received.
Raw response: b''
Failed to parse response: unpack requires a buffer of 12 bytes
No A record found or an error occurred.


2. What are the drawbacks of DoH and DoT, and how could these drawbacks be addressed?

DNS over HTTPS (DoH) and DNS over TLS (DoT) introduce several drawbacks. They can face middlebox interference from network devices like firewalls, which may not properly handle or might block encrypted DNS traffic. Additionally, encrypting DNS queries and responses can increase latency due to the added computational overhead. There's also a risk of centralization with DoH, as it could consolidate DNS traffic to a few major providers, potentially reducing diversity in DNS resolution options. Operational challenges include the complexity of deploying and maintaining encrypted DNS services, especially for smaller operators.

These drawbacks can be addressed by promoting standards adoption and improving compatibility with existing network infrastructure to mitigate middlebox interference. Continual optimization of encryption algorithms and implementation efficiencies can help reduce latency. Encouraging a diverse ecosystem of DNS resolver providers offering DoH and DoT services can mitigate centralization risks and foster competition. Providing education and resources to support the deployment and management of DoH and DoT can help address operational challenges and ensure broader adoption across different network environments.